In [ ]:
import pandas as pd
import math 

In [ ]:
## 
# Load the treeinfo file to a Pandas DataFrame
## 


def treeinfo_attributes_segment(tree_file):
    """
    Extracts per-segment attributes of a tree file generated using treeinfo and returns a DataFrame.
    Can be run on a 'forest' or single treefile.

    Parameters:
    tree_file (str): The path to the treefile created using treeinfo.

    Returns:
    pandas.DataFrame: A DataFrame containing segment attributes. If the input file is a forest, the DataFrame will contain a column 'tree_id' with the tree ID.
    """
    line_list = []
    tree_ids = []
    tree_id = 0
    
    with open(tree_file, 'r') as file:
        lines = file.readlines()
        line_count = 0        
        for line in lines:
            data = line.split(', ')
            for row in data:
                section_data = row.strip().split(', ')
                cell_data = section_data[0].strip().split(',')
                if len(cell_data) == 7 and all(x.replace('.', '', 1).isdigit() for x in cell_data):
                    tree_id += 1
                if len(cell_data) > 7:
                    if tree_id != 0:
                        tree_ids.append(tree_id)
                    line_list.append(cell_data)
            line_count += 1
    df = pd.DataFrame(line_list[1:], columns=line_list[0]).astype(float)
    df.insert(0, 'tree_id', tree_ids)
    # remove row where parent_id is -1.0
    df = df[df['parent_id'] != -1.0]
    return df

df = treeinfo_attributes_segment('example_data/tree_info.txt')
df

segment_df
    

In [ ]:
## 
# Adjust the start and end coordinates of each segment
## 

# Iterate over each node to set the correct start coordinates
for idx, row in df.iterrows():
    pid = row['parent_id']
    if pid == 0 or pd.isnull(pid):
        # Root node: start coordinates are the same as current node, but z is adjusted by segment length
        df.at[idx, 'start_x'] = row['x']
        df.at[idx, 'start_y'] = row['y']
        df.at[idx, 'start_z'] = row['z'] - row['segment_length']
    else:
        # Ensure parent_id is an integer
        pid = int(pid)
        # Retrieve parent's coordinates
        parent_row = df.loc[pid]
        df.at[idx, 'start_x'] = parent_row['x']
        df.at[idx, 'start_y'] = parent_row['y']
        df.at[idx, 'start_z'] = parent_row['z']

# Change x,y,z to start_x, start_y, start_z
df = df.rename(columns={'x': 'end_x', 'y': 'end_y', 'z': 'end_z'})

# Display the updated DataFrame
df

In [ ]:
##
# Calculate the direction vector for each segment
## 

def calculate_direction_vector(start_point, end_point):
    """
    Calculate the direction vector from start_point to end_point in 3D space.
    
    :param start_point: A tuple or list containing (x, y, z) coordinates of the start point
    :param end_point: A tuple or list containing (x, y, z) coordinates of the end point
    :return: A tuple containing the direction vector (dx, dy, dz)
    """
    dx = end_point[0] - start_point[0]
    dy = end_point[1] - start_point[1]
    dz = end_point[2] - start_point[2]
    
    # Normalize the vector (optional)
    magnitude = math.sqrt(dx**2 + dy**2 + dz**2)
    if magnitude != 0:
        dx /= magnitude
        dy /= magnitude
        dz /= magnitude
    
    return (round(dx, 5), round(dy, 5), round(dz, 5))

# Calculate the direction vector for each segment, and save as 3 precision float
df['direction_vector'] = df.apply(lambda row: calculate_direction_vector((row['start_x'], row['start_y'], row['start_z']), (row['end_x'], row['end_y'], row['end_z'])), axis=1)

# Display the updated DataFrame
df

In [ ]:
# Write the DataFrame to a CSV file
df.to_csv('tree_info_parsed.csv', index=False)

In [ ]:
import open3d as o3d
import numpy as np
import pandas as pd

def rotation_matrix_from_vectors(vec1, vec2):
    """Find the rotation matrix that aligns vec1 to vec2."""
    a = vec1 / np.linalg.norm(vec1) if np.linalg.norm(vec1) > 0 else np.array([1, 0, 0])
    b = vec2 / np.linalg.norm(vec2) if np.linalg.norm(vec2) > 0 else np.array([1, 0, 0])

    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)

    # Handle collinear vectors
    if s == 0:
        # If vectors are in the same or opposite direction
        return np.eye(3) if c > 0 else -np.eye(3)

    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix


def reconstruct_treefile(tree_df):
    """
    Reconstructs a tree from a treefile and returns a TriangleMesh.
    """
    mesh = o3d.geometry.TriangleMesh()
    for index, row in tree_df.iterrows():
        try:
            # Validate segment_length and radius
            if row['segment_length'] <= 0 or row['radius'] <= 0:
                print(f"Skipping row {index}: Invalid segment_length or radius.")
                continue

            # Create a mesh cylinder
            cylinder = o3d.geometry.TriangleMesh.create_cylinder(
                radius=row['radius'], 
                height=row['segment_length'], 
                resolution=8, 
                split=4
            )

            # Align the cylinder with the direction vector
            z_axis = np.array([0, 0, 1])
            direction_vector = np.array(row['direction_vector'])
            R = rotation_matrix_from_vectors(z_axis, direction_vector)
            cylinder.rotate(R, center=(0, 0, 0))

            # Translate the cylinder to the correct position
            cylinder.translate((row['start_x'], row['start_y'], row['start_z']))

            # Combine with the main mesh
            mesh += cylinder
        except Exception as e:
            print(f"Error processing row {index}: {e}")
    return mesh

# Example DataFrame (replace this with your actual DataFrame)
# df = pd.read_csv("your_data.csv") or similar

# Reconstruct the tree mesh
tree_mesh = reconstruct_treefile(df)

# Save the tree mesh to a file
if len(tree_mesh.vertices) > 0:
    o3d.io.write_triangle_mesh('tree_mesh.ply', tree_mesh)
    # Visualize the tree
    o3d.visualization.draw_geometries([tree_mesh])
else:
    print("No valid mesh generated.")
